<a href="https://colab.research.google.com/github/BlueSky2311/NAMD/blob/main/namd_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Hướng dẫn:
!!!! Luôn chỉnh đường dẫn file mỗi khi sử dụng
- Lưu file zip vào GDrive
- Extract từ file zip từ GDrive (sau khi import GDrive trỏ đường dẫn file zip trong drive vào địa chỉ ở cell 2)
- Nếu là lần đầu thì nên copy thư mục vừa giải nén vào GDrive
- Sửa đường dẫn file input và ouput ở cell 4
- Chạy NAMD

* Trường hợp disconnect giữa chừng thì reconnect lại và copy các file cần thiết để chạy lại: *.log *.coor *.vel *.xsc *.xst *.dcd
Mở file .xst lên xem đang chạy tới bao nhiêu, thêm dòng Firtsttimestep với giá trị = số step đang chạy mà bị dừng
Mở file .inp: sửa tên file bin, output và numstep/run

In [ ]:
#@title Load google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Download and prepare the charmm-gui files
import pandas as pd
import os
import shutil

# Load the CSV file
df = pd.read_csv('/path/to/your/file.csv')

# Get the jobids from the first column
jobids = df.iloc[:, 0].tolist()

# Define a list to store the names of the extracted folders
extracted_folders = []

# Loop over each jobid
for jobid in jobids:
    # Download the CHARMM-GUI files
    !curl 'https://www.charmm-gui.org/?doc=input/download&jobid={jobid}&compress_only=1' \
    -H 'Accept: */*' \
    -H 'Accept-Language: en-US,en;q=0.9' \
    -H 'Connection: keep-alive' \
    -H 'Cookie: phpbb3_8fjan_k=; amp_6e403e=nuofvFynNKI6uifCRx6ccn...1h3u0uikt.1h3u0uimq.0.bf.bf; PHPSESSID=973jb8lvtida0ifteedv4eusdt; phpbb3_8fjan_u=30723; phpbb3_8fjan_sid=641f1aee61ee298947da31badb25a25c; visit=ok; SL_G_WPT_TO=en; SL_GWPT_Show_Hide_tmp=1; SL_wptGlobTipTmp=1' \
    -H 'DNT: 1' \
    -H 'Referer: https://www.charmm-gui.org/?doc=input/solution&step=4&project=solution&jobid={jobid}' \
    -H 'Sec-Fetch-Dest: empty' \
    -H 'Sec-Fetch-Mode: cors' \
    -H 'Sec-Fetch-Site: same-origin' \
    -H 'User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0' \
    -H 'sec-ch-ua: "Microsoft Edge";v="123", "Not:A-Brand";v="8", "Chromium";v="123"' \
    -H 'sec-ch-ua-mobile: ?0' \
    -H 'sec-ch-ua-platform: "Windows"' -o charmm-gui.tgz

    # Extract the CHARMM-GUI files
    !tar -zxf charmm-gui.tgz

    # Define the name of the extracted folder
    extracted_folder = f"charmm-gui-{jobid}"

    # Add the name of the extracted folder to the list
    extracted_folders.append(extracted_folder)

    # Remove the downloaded .tgz file
    os.remove("charmm-gui.tgz")

# Move the extracted folders to /content/drive/MyDrive/namd after all the downloads and extractions are done
for folder in extracted_folders:
    shutil.move(folder, f"/content/drive/MyDrive/namd/{folder}")


In [ ]:
#@title Install NAMD
# Download the NAMD
!wget https://www.ks.uiuc.edu/Research/namd/3.0b5/download/019723/NAMD_3.0b5_Linux-x86_64-multicore-CUDA.tar.gz
#Extract NAMD
!tar -zxf /content/NAMD_3.0b5_Linux-x86_64-multicore-CUDA.tar.gz

In [ ]:
#@title Run NAMD

# Get the list of all directories in /MyDrive/namd
folders = os.listdir('/content/drive/MyDrive/namd')

# Loop over each folder
for folder in folders:
    # Construct the file path
    file_path = f"/content/drive/MyDrive/namd/{folder}/{folder}/namd/step4_equilibration.inp"
    
    # Run the NAMD simulation
    !/content/NAMD_3.0b4_Linux-x86_64-multicore-CUDA.tar.gz +auto-provision +idlepoll +devices 0 {file_path} > /content/drive/MyDrive/namd/{folder}/{folder}/namd/step4_equilibration.log &


In [ ]:
#@title Copy file to GDrive
#ls /content/drive/MyDrive/namd/B1/B1/namd
!cp *.log *.coor *.vel *.xsc *.xst *.dcd /content/drive/MyDrive

To combine multiples .dcd files, see this notebook:https://colab.research.google.com/github/BlueSky2311/NAMD/blob/main/CatDCD.ipynb

References:

1. https://www.youtube.com/watch?v=NHb5s9zDZP8
2. James C. Phillips, David J. Hardy, Julio D. C. Maia, John E. Stone, João V. Ribeiro, Rafael C. Bernardi, Ronak Buch, Giacomo Fiorin, Jérôme Hénin, Wei Jiang, Ryan McGreevy, Marcelo C. R. Melo, Brian Radak, Robert D. Skeel, Abhishek Singharoy, Yi Wang, Benoît Roux, Aleksei Aksimentiev, Zaida Luthey-Schulten, Laxmikant V. Kalé, Klaus Schulten, Christophe Chipot, and Emad Tajkhorshid. Scalable molecular dynamics on CPU and GPU architectures with NAMD. Journal of Chemical Physics, 153:044130, 2020. (PMC: PMC7395834)

